<a href="https://colab.research.google.com/github/Milantamang1/ChatbotUsingTf-idfCosineSimilarity/blob/main/ProductionChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
# Load the knowledge base
df = pd.read_excel('/content/drive/MyDrive/ChatbotRefined.xlsx')
df.fillna('', inplace=True)
print(df.head())

            QuestionNepali               QuestionRoman   QuestionEnglish  \
0              तिमी को हौ?                Timi Ko hau?      Who are you?   
1              के सक्छौ त?               K sakchau ta?  What can you do?   
2         तिमीलाई कस्तो छ?          Timilai kasto cha?      How are you?   
3  तिमीलाई कसले बनाएको हो?  Timilai kasle banayeko ho?     Who made you?   
4           तिमी के गर्छौ?             Timi k garchau?  What can you do?   

                                              Answer Image_path  
0                    म Axis ले बनाएको च्याटबोट हुँ |             
1  म विभिन्न विषयमा प्रश्नहरूको उत्तर दिन सक्छु —...             
2             म ठिक छु, धन्यवाद! तपाईंलाई कस्तो छ त?             
3                    म Axis ले बनाएको च्याटबोट हुँ |             
4                         म प्रश्नहरूको जवाफ दिन्छु।             


In [ ]:
# Create a unified, normalized column
df['Combined_Question'] = (
    df['QuestionNepali'].str.lower() + ' ' +
    df['QuestionRoman'].str.lower() + ' ' +
    df['QuestionEnglish'].str.lower()
)
print(df[['Combined_Question']].head())

                                   Combined_Question
0              तिमी को हौ? timi ko hau? who are you?
1         के सक्छौ त? k sakchau ta? what can you do?
2   तिमीलाई कस्तो छ? timilai kasto cha? how are you?
3  तिमीलाई कसले बनाएको हो? timilai kasle banayeko...
4    तिमी के गर्छौ? timi k garchau? what can you do?


In [ ]:
import unicodedata
def normalize_input(user_input):
    return unicodedata.normalize("NFKC", user_input.strip().lower())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Build TF-IDF on the entire dataset
vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\b\w+\b')
corpus = df['Combined_Question'].tolist()
tfidf_matrix = vectorizer.fit_transform(corpus)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Image, display
import pandas as pd
import unicodedata
import os
import re

# ✅ Load the dataset
df = pd.read_excel("/content/drive/MyDrive/ChatbotRefined.xlsx")
df.fillna("", inplace=True)

# ✅ Combine all languages for TF-IDF
df['Combined_Question'] = (
    df['QuestionNepali'].str.lower().str.strip() + " " +
    df['QuestionRoman'].str.lower().str.strip() + " " +
    df['QuestionEnglish'].str.lower().str.strip()
)

# ✅ Normalize input (Unicode-safe, preserves Nepali)
def normalize_input(text):
    text = text.lower().strip()
    text = unicodedata.normalize("NFKC", text)  # Normalize full-width & Nepali characters
    return re.sub(r"\s+", " ", text)  # Collapse multiple spaces

# ✅ Build TF-IDF matrix
vectorizer = TfidfVectorizer(analyzer="word", token_pattern=r"\b\w+\b")
corpus = df["Combined_Question"].tolist()
tfidf_matrix = vectorizer.fit_transform(corpus)

# ✅ Path to images in Google Drive
BASE_IMAGE_PATH = "/content/drive/MyDrive/Images"

# ✅ Get best matching answer
def get_best_answer(query, threshold=0.4):
    query_norm = normalize_input(query)
    query_tfidf = vectorizer.transform([query_norm])
    cos_scores = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    best_index = cos_scores.argmax()
    best_score = cos_scores[best_index]

    print(f"\n🔎 Similarity Score: {best_score:.2f}")

    if best_score < threshold:
        return "माफ गर्नुहोस्, तपाईंको प्रश्न राम्रोसँग बुझ्न सकिनँ।", None

    best_answer = df.iloc[best_index]["Answer"]
    image_path = df.iloc[best_index]["Image_path"]
    return best_answer, image_path

# ✅ Display image
def show_image(image_path):
    if not isinstance(image_path, str) or not image_path.strip():
        print("🖼️ No image provided.")
        return

    image_filename = image_path.replace("Images/", "").strip()
    full_path = os.path.join(BASE_IMAGE_PATH, image_filename)

    if os.path.exists(full_path):
        display(Image(filename=full_path))
    else:
        print(f"⚠️ Image not found: {full_path}")

# ✅ Chat loop
print("🤖 Chatbot is ready! (Type 'exit' to quit)")
while True:
    query = input("\nYou: ").strip()
    if query.lower() in ["exit", "quit", "stop"]:
        print("👋 Exiting chatbot. Goodbye!")
        break

    answer, image = get_best_answer(query)
    print("Bot:", answer)
    if image:
        show_image(image)


🤖 Chatbot is ready! (Type 'exit' to quit)

You: तिमीलाई कस्तो प्रश्न सोध्न मिल्छ?

🔎 Similarity Score: 0.65
Bot: मलाई यो सफ़्टोयर को सम्पूर्ण जानकारी सोध्न सक्छौ। 

You: तिमी के गर्छौ?

🔎 Similarity Score: 0.49
Bot: म प्रश्नहरूको जवाफ दिन्छु।

You: Timi k garchau?

🔎 Similarity Score: 0.63
Bot: म प्रश्नहरूको जवाफ दिन्छु।

You: What kind of question can I ask you?

🔎 Similarity Score: 0.58
Bot: मलाई यो सफ़्टोयर को सम्पूर्ण जानकारी सोध्न सक्छौ। 

You: exit
👋 Exiting chatbot. Goodbye!


In [ ]:
# def get_best_answer(query, threshold=0.4):
#     query_norm = normalize_input(query)
#     query_tfidf = vectorizer.transform([query_norm])
#     cos_scores = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

#     best_index = cos_scores.argmax()
#     best_score = cos_scores[best_index]

#     print(f"Similarity Score: {best_score:.2f}")

#     if best_score < threshold:
#         return "माफ गर्नुहोस्, तपाईंको प्रश्न राम्रोसँग बुझ्न सकिनँ।", None

#     best_answer = df.iloc[best_index]['Answer']
#     image_path = df.iloc[best_index]['Image_path']

#     return best_answer, image_path
# # def get_best_answer(query, session_context, threshold=0.4):
# #     # Check for memory reset
# #     if query.lower().startswith("new topic:"):
# #         session_context.clear()
# #         query = query[len("new topic:"):].strip()

# #     # Normalize and expand query using context
# #     query_norm = normalize_input(query)
# #     full_query = " ".join(session_context[-2:] + [query_norm])  # last 2 + current
# #     full_query_tfidf = vectorizer.transform([full_query])

# #     cos_scores = cosine_similarity(full_query_tfidf, tfidf_matrix).flatten()
# #     best_index = cos_scores.argmax()
# #     best_score = cos_scores[best_index]

# #     print(f"Similarity Score: {best_score:.2f}")

# #     if best_score < threshold:
# #         return "माफ गर्नुहोस्, तपाईंको प्रश्न राम्रोसँग बुझ्न सकिनँ।", None

# #     # Update context only after match
# #     session_context.append(query_norm)

# #     best_answer = df.iloc[best_index]['Answer']
# #     image_path = df.iloc[best_index]['Image_path']
# #     return best_answer, image_path


In [ ]:
# from IPython.display import Image, display
# import os

# # Define the base path where all images are stored
# BASE_IMAGE_PATH = "/content/drive/MyDrive/Images"

# def show_image(image_path):
#     """
#     Displays an image if a valid image path is provided and exists.
# """
#     # Check if the image path is a valid non-empty string
#     if not isinstance(image_path, str) or not image_path.strip():
#         print("No valid image to show.")
#         return

#     # Clean the path
#     image_path = image_path.strip()

#     # Remove 'Images/' prefix if present
#     image_filename = image_path.replace("Images/", "")

#     # Construct the full path to the image
#     full_path = os.path.join(BASE_IMAGE_PATH, image_filename)

#     # Display if exists, otherwise show warning
#     if os.path.exists(full_path):
#         display(Image(filename=full_path))
#     else:
#         print(f"⚠️ Image file not found: {full_path}")



In [ ]:
# # To store previous questions for context-aware retrieval
# session_context = []

In [ ]:
# while True:
#     query = input("You: ")
#     if query.lower().strip() in ['exit', 'quit', 'stop']:
#         print("Exiting chatbot")
#         break
#     answer, image = get_best_answer(query)
#     print("Bot:", answer)
#     if image:
#         show_image(image)
# while True:
#     query = input("You: ").strip()
#     if query.lower() in ['exit', 'quit', 'stop']:
#         print("Exiting chatbot. 👋")
#         break

#     answer, image = get_best_answer(query, session_context)
#     print("Bot:", answer)

#     if image:
#         show_image(image)
